In [1]:
import sys
import pathlib
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
import plotly.express as px
from pycytominer.cyto_utils import infer_cp_features
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import pdfkit
import pyarrow as pa
import pyarrow.parquet as pq

# import Union
from typing import Union

sys.path.append("..")
# from ..utils.utils import df_stats
import matplotlib.pyplot as plt

In [2]:
# Define inputs
feature_file = pathlib.Path(
    "../../Extracted_Features_(CSV_files)/SHSY5Y_run_sc_norm.parquet"
)
feature_df = pq.read_table(feature_file).to_pandas()
# feature_df = pd.read_csv(feature_file, engine="pyarrow")

In [3]:
# define output file path
feature_df_out_path = pathlib.Path(
    "../../Extracted_Features_(CSV_files)/feature_df_sc_norm.parquet"
)

In [4]:
print(feature_df.shape)
feature_df

(597902, 2926)


,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_Variance_CorrGasdermin_3_02_256,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,0.011881,0.003817,0.353469,0.365618,0.378125,0.342100,0.099225,0.096054,0.110432,0.107764
1,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.044113,-0.042322,1.780397,1.603914,1.571446,1.582787,0.020683,0.016122,0.012990,0.013631
2,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.051521,-0.054760,0.019005,0.041005,0.029673,0.007339,0.027519,0.045982,0.037653,0.017281
3,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.058159,-0.059141,-0.005212,-0.010673,0.000258,0.003475,-0.002945,0.000293,0.008048,-0.002151
4,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.034966,-0.030255,-0.044078,-0.052547,-0.042483,-0.029738,0.016846,0.006499,0.014114,0.026206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597897,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,-0.070301,-0.068560,-0.117209,-0.125985,-0.127684,-0.122355,-0.024370,-0.033577,-0.033811,-0.028148
597898,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,0.000763,-0.007537,-0.108656,-0.110885,-0.112064,-0.113660,-0.049855,-0.048752,-0.050176,-0.049417
597899,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,-0.049428,-0.050434,-0.088444,-0.089816,-0.092630,-0.092441,-0.027676,-0.024066,-0.026279,-0.027050
597900,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,0.078521,0.081602,0.098873,0.110850,0.109012,0.112935,0.308702,0.309290,0.311365,0.309627


In [5]:
# removing costes features as they behave with great variance across all data
feature_df = feature_df.drop(feature_df.filter(regex="Costes").columns, axis=1)
print(feature_df.shape)
feature_df

(597902, 2866)


,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_Variance_CorrGasdermin_3_02_256,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,0.011881,0.003817,0.353469,0.365618,0.378125,0.342100,0.099225,0.096054,0.110432,0.107764
1,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.044113,-0.042322,1.780397,1.603914,1.571446,1.582787,0.020683,0.016122,0.012990,0.013631
2,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.051521,-0.054760,0.019005,0.041005,0.029673,0.007339,0.027519,0.045982,0.037653,0.017281
3,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.058159,-0.059141,-0.005212,-0.010673,0.000258,0.003475,-0.002945,0.000293,0.008048,-0.002151
4,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.034966,-0.030255,-0.044078,-0.052547,-0.042483,-0.029738,0.016846,0.006499,0.014114,0.026206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597897,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,-0.070301,-0.068560,-0.117209,-0.125985,-0.127684,-0.122355,-0.024370,-0.033577,-0.033811,-0.028148
597898,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,0.000763,-0.007537,-0.108656,-0.110885,-0.112064,-0.113660,-0.049855,-0.048752,-0.050176,-0.049417
597899,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,-0.049428,-0.050434,-0.088444,-0.089816,-0.092630,-0.092441,-0.027676,-0.024066,-0.026279,-0.027050
597900,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,0.078521,0.081602,0.098873,0.110850,0.109012,0.112935,0.308702,0.309290,0.311365,0.309627


In [6]:
# replacing '/' in treatment dosage column to avoid errors in file interpolation including such strings
feature_df = feature_df.replace(to_replace="/", value="_per_", regex=True)
feature_df

,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_Variance_CorrGasdermin_3_02_256,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,0.011881,0.003817,0.353469,0.365618,0.378125,0.342100,0.099225,0.096054,0.110432,0.107764
1,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.044113,-0.042322,1.780397,1.603914,1.571446,1.582787,0.020683,0.016122,0.012990,0.013631
2,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.051521,-0.054760,0.019005,0.041005,0.029673,0.007339,0.027519,0.045982,0.037653,0.017281
3,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.058159,-0.059141,-0.005212,-0.010673,0.000258,0.003475,-0.002945,0.000293,0.008048,-0.002151
4,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.034966,-0.030255,-0.044078,-0.052547,-0.042483,-0.029738,0.016846,0.006499,0.014114,0.026206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597897,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,-0.070301,-0.068560,-0.117209,-0.125985,-0.127684,-0.122355,-0.024370,-0.033577,-0.033811,-0.028148
597898,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,0.000763,-0.007537,-0.108656,-0.110885,-0.112064,-0.113660,-0.049855,-0.048752,-0.050176,-0.049417
597899,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,-0.049428,-0.050434,-0.088444,-0.089816,-0.092630,-0.092441,-0.027676,-0.024066,-0.026279,-0.027050
597900,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,0.078521,0.081602,0.098873,0.110850,0.109012,0.112935,0.308702,0.309290,0.311365,0.309627


In [7]:
# Recycled code from: https://github.com/WayScience/NF1_SchwannCell_data/blob/main/5_analyze_data/notebooks/linear_model/fit_linear_model.ipynb
cell_count_df = (
    feature_df.groupby("Metadata_Well")["Metadata_Plate"]
    .count()
    .reset_index()
    .rename(columns={"Metadata_Plate": "Metadata_number_of_singlecells"})
)

In [8]:
feature_df

,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_Variance_CorrGasdermin_3_02_256,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,0.011881,0.003817,0.353469,0.365618,0.378125,0.342100,0.099225,0.096054,0.110432,0.107764
1,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.044113,-0.042322,1.780397,1.603914,1.571446,1.582787,0.020683,0.016122,0.012990,0.013631
2,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.051521,-0.054760,0.019005,0.041005,0.029673,0.007339,0.027519,0.045982,0.037653,0.017281
3,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.058159,-0.059141,-0.005212,-0.010673,0.000258,0.003475,-0.002945,0.000293,0.008048,-0.002151
4,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.034966,-0.030255,-0.044078,-0.052547,-0.042483,-0.029738,0.016846,0.006499,0.014114,0.026206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597897,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,-0.070301,-0.068560,-0.117209,-0.125985,-0.127684,-0.122355,-0.024370,-0.033577,-0.033811,-0.028148
597898,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,0.000763,-0.007537,-0.108656,-0.110885,-0.112064,-0.113660,-0.049855,-0.048752,-0.050176,-0.049417
597899,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,-0.049428,-0.050434,-0.088444,-0.089816,-0.092630,-0.092441,-0.027676,-0.024066,-0.026279,-0.027050
597900,SH-SY5Y,O23,3555,6,Media ctr,NaN,None,media ctr,None,None,...,0.078521,0.081602,0.098873,0.110850,0.109012,0.112935,0.308702,0.309290,0.311365,0.309627


In [9]:
# show max column in pandas df
pd.set_option("display.max_columns", 100)

In [10]:
# replace nan values with 0
feature_df["Metadata_inducer1_concentration"] = feature_df[
    "Metadata_inducer1_concentration"
].fillna(0)
feature_df["Metadata_inducer2_concentration"] = feature_df[
    "Metadata_inducer2_concentration"
].fillna(0)
feature_df["Metadata_inhibitor_concentration"] = feature_df[
    "Metadata_inhibitor_concentration"
].fillna(0)

In [11]:
feature_df["Metadata_inducer2"].unique()

array([None, 'Nigericin'], dtype=object)

In [12]:
feature_df

,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,Metadata_inducer2,Metadata_inducer2_concentration,Metadata_inducer2_concentration_unit,Metadata_ImageNumber,Metadata_Plate,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Nuclei_Number_Object_Number,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_BoundingBoxMinimum_Y,Cytoplasm_AreaShape_Center_X,Cytoplasm_AreaShape_Center_Y,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_ConvexArea,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_EquivalentDiameter,Cytoplasm_AreaShape_EulerNumber,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MaxFeretDiameter,Cytoplasm_AreaShape_MaximumRadius,Cytoplasm_AreaShape_MeanRadius,Cytoplasm_AreaShape_MedianRadius,Cytoplasm_AreaShape_MinFeretDiameter,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Perimeter,Cytoplasm_AreaShape_Solidity,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,Cytoplasm_AreaShape_Zernike_3_1,Cytoplasm_AreaShape_Zernike_3_3,...,Nuclei_Texture_SumEntropy_CorrGasdermin_3_02_256,Nuclei_Texture_SumEntropy_CorrGasdermin_3_03_256,Nuclei_Texture_SumEntropy_CorrMito_3_00_256,Nuclei_Texture_SumEntropy_CorrMito_3_01_256,Nuclei_Texture_SumEntropy_CorrMito_3_02_256,Nuclei_Texture_SumEntropy_CorrMito_3_03_256,Nuclei_Texture_SumEntropy_CorrPM_3_00_256,Nuclei_Texture_SumEntropy_CorrPM_3_01_256,Nuclei_Texture_SumEntropy_CorrPM_3_02_256,Nuclei_Texture_SumEntropy_CorrPM_3_03_256,Nuclei_Texture_SumVariance_CorrDNA_3_00_256,Nuclei_Texture_SumVariance_CorrDNA_3_01_256,Nuclei_Texture_SumVariance_CorrDNA_3_02_256,Nuclei_Texture_SumVariance_CorrDNA_3_03_256,Nuclei_Texture_SumVariance_CorrER_3_00_256,Nuclei_Texture_SumVariance_CorrER_3_01_256,Nuclei_Texture_SumVariance_CorrER_3_02_256,Nuclei_Texture_SumVariance_CorrER_3_03_256,Nuclei_Texture_SumVariance_CorrGasdermin_3_00_256,Nuclei_Texture_SumVariance_CorrGasdermin_3_01_256,Nuclei_Texture_SumVariance_CorrGasdermin_3_02_256,Nuclei_Texture_SumVariance_CorrGasdermin_3_03_256,Nuclei_Texture_SumVariance_CorrMito_3_00_256,Nuclei_Texture_SumVariance_CorrMito_3_01_256,Nuclei_Texture_SumVariance_CorrMito_3_02_256,Nuclei_Texture_SumVariance_CorrMito_3_03_256,Nuclei_Texture_SumVariance_CorrPM_3_00_256,Nuclei_Texture_SumVariance_CorrPM_3_01_256,Nuclei_Texture_SumVariance_CorrPM_3_02_256,Nuclei_Texture_SumVariance_CorrPM_3_03_256,Nuclei_Texture_Variance_CorrDNA_3_00_256,Nuclei_Texture_Variance_CorrDNA_3_01_256,Nuclei_Texture_Variance_CorrDNA_3_02_256,Nuclei_Texture_Variance_CorrDNA_3_03_256,Nuclei_Texture_Variance_CorrER_3_00_256,Nuclei_Texture_Variance_CorrER_3_01_256,Nuclei_Texture_Variance_CorrER_3_02_256,Nuclei_Texture_Variance_CorrER_3_03_256,Nuclei_Texture_Variance_CorrGasdermin_3_00_256,Nuclei_Texture_Variance_CorrGasdermin_3_01_256,Nuclei_Texture_Variance_CorrGasdermin_3_02_256,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0,None,None,0.0,None,1,*70117_20230210MM1_Gasdermin514_CP_BC430856,1,1,3,3,-0.947035,-0.688884,1.399821,-1.742919,1.476026,-1.723436,1.432244,-1.725772,0.948831,-1.007165,1.115044,-1.231865,0.02

In [13]:
(
    feature_df["Metadata_inducer1"]
    + "_"
    + feature_df["Metadata_inducer1_concentration"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor"]
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
    + "_"
    + feature_df["Metadata_inducer2"]
    + "_"
    + feature_df["Metadata_inducer2_concentration"].astype(str)
).unique()


(
    feature_df["Metadata_inducer1"]
    + "_"
    + feature_df["Metadata_inducer1_concentration"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor"]
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
).unique()

array(['media ctr_0_Media ctr_0.0', 'DMSO_0.100_DMSO_1.0',
       'DMSO_0.100_Z-VAD-FMK_100.0', 'DMSO_0.100_Z-VAD-FMK_30.0',
       'DMSO_0.100_DMSO_0.025', 'Thapsigargin_1.000_DMSO_0.025',
       'Thapsigargin_10.000_DMSO_0.025', 'Topotecan_5.000_DMSO_0.025',
       'Topotecan_10.000_DMSO_0.025', 'Topotecan_20.000_DMSO_0.025',
       'LPS_0.010_DMSO_0.025', 'LPS_0.100_DMSO_0.025',
       'LPS_1.000_DMSO_0.025', 'LPS_10.000_DMSO_0.025',
       'LPS_10.000_Disulfiram_0.1', 'LPS_10.000_Disulfiram_1.0',
       'LPS_10.000_Disulfiram_2.5', 'LPS_100.000_DMSO_0.025',
       'Disulfiram_0.100_DMSO_0.025', 'Disulfiram_1.000_DMSO_0.025',
       'Disulfiram_2.500_DMSO_0.025', 'H2O2_100.000_DMSO_0.025',
       'LPS_10.000_Z-VAD-FMK_100.0', 'LPS_1.000_Disulfiram_1.0',
       'LPS_1.000_Z-VAD-FMK_100.0', 'H2O2_100.000_Disulfiram_1.0',
       'H2O2_100.000_Z-VAD-FMK_100.0', 'Flagellin_0.100_DMSO_0.025',
       'Flagellin_1.000_DMSO_0.025', 'Flagellin_1.000_Disulfiram_1.0'],
      dtype=object)

In [14]:
# feature_df = feature_df.merge(cell_count_df, on="Metadata_Well")

# Drop na and reindex accordingly
feature_df = feature_df.assign(
    Metadata_Treatment_and_Dose=lambda x: feature_df["Metadata_inducer1"]
    + "_"
    + feature_df["Metadata_inducer1_concentration"].astype(str)
)

We are testing 2847 CellProfiler features
The unique Treatment-Dosages are: media ctr_0, 
DMSO_0.100, 
Thapsigargin_1.000, 
Thapsigargin_10.000, 
Topotecan_5.000, 
Topotecan_10.000, 
Topotecan_20.000, 
LPS_0.010, 
LPS_0.100, 
LPS_1.000, 
LPS_10.000, 
LPS_100.000, 
Disulfiram_0.100, 
Disulfiram_1.000, 
Disulfiram_2.500, 
H2O2_100.000, 
Flagellin_0.100, 
Flagellin_1.000


In [20]:
feature_df["Metadata_inducer1_concentration"] = feature_df[
    "Metadata_inducer1_concentration"
].astype(float)
feature_df["Metadata_inducer1_concentration"].unique()

array([0.0e+00, 1.0e-01, 1.0e+00, 1.0e+01, 5.0e+00, 2.0e+01, 1.0e-02,
       1.0e+02, 2.5e+00])

In [21]:
# feature_df.to_csv(feature_df_out_path, index=False)
feature_df_table = pa.Table.from_pandas(feature_df)
pq.write_table(feature_df_table, feature_df_out_path)